# [stack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html)
- 컬럼명을 index(행명)으로 전환
    - 기존 index가 있으면 하위 레벨로 들어간다. (기존 것이 상위 레벨)
- 컬럼명을 컬럼의 값으로 전치시킬때도 사용할 수 있다.
- 매개변수
    - dropna=False : stacking 시 생성되는 NA(결측치)는 제거되지 않게 한다. (default: True => 제거)


In [1]:
import pandas as pd
import numpy as np

In [4]:
data=dict(State=['Texas','Arizona','Florida'],
         Apple=[12,13,14],
         Orange=[9,7,5],
         Banana=[40,35,52])
state_fruits=pd.DataFrame(data).set_index('State')
state_fruits

,Apple,Orange,Banana
State,,,
Texas,12,9,40
Arizona,13,7,35
Florida,14,5,52


In [5]:
# 컬럼명이 컬럼의 값 또는 index가 돼야 한다. -> 각 셀의 값들이 무엇을 의미하는지 알 수 없다.

In [9]:
s1=state_fruits.stack()  # 컬럼명을 index로 변환
s1.reset_index()  # index를 column의 값으로 뺀다

,State,level_1,0
0,Texas,Apple,12
1,Texas,Orange,9
2,Texas,Banana,40
3,Arizona,Apple,13
4,Arizona,Orange,7
5,Arizona,Banana,35
6,Florida,Apple,14
7,Florida,Orange,5
8,Florida,Banana,52


In [10]:
s1['Texas','Apple']

12

### rename_axis()
- index명이나 columns명에 label을 붙인다. (메타데이터 같은 역할)
- 매개변수
    - mapper: 지정할 이름
    - axis : 이름을 붙일 축 지정 (0 - index명의 label(기본-시리즈는 이것만), 1-column명의 label) 
    - columns, index : 직접 이름 지정. 한번에 두개 다 바꿀때
```python
state_fruit.rename_axis('Test', axis=0)
```

In [13]:
s1.rename_axis(['State','Fruits']).reset_index(name='Count')

,State,Fruits,Count
0,Texas,Apple,12
1,Texas,Orange,9
2,Texas,Banana,40
3,Arizona,Apple,13
4,Arizona,Orange,7
5,Arizona,Banana,35
6,Florida,Apple,14
7,Florida,Orange,5
8,Florida,Banana,52


### reset_index()
- Series.reset_index(name='열이름')
    - Series 경우, reset_index하면 index가 데이터셋으로 나오면서 DataFrame이 된다.
    - 이때 value의 컬럼명을 name매개변수로 지정한다. (이건 **Series일때만 가능-DF는 안된다.**)

In [18]:
s2=s1.reset_index()
s2.columns=['주','과일명','개수']
s2

,주,과일명,개수
0,Texas,Apple,12
1,Texas,Orange,9
2,Texas,Banana,40
3,Arizona,Apple,13
4,Arizona,Orange,7
5,Arizona,Banana,35
6,Florida,Apple,14
7,Florida,Orange,5
8,Florida,Banana,52


# unstack()
- stack() 반대로 index를 컬럼으로 변환한다.
- 매개변수 
    - level: multi-index일 경우 컬럼으로 만들 레벨을 지정한다. 기본값은 -1 로 가장 안쪽의 index를 이동시킨다.

In [19]:
df=s2.set_index(['주','과일명'])
df

개수
주       과일명       
Texas   Apple   12
        Orange   9
        Banana  40
Arizona Apple   13
        Orange   7
        Banana  35
Florida Apple   14
        Orange   5
        Banana  52

In [20]:
df.unstack()

개수              
과일명     Apple Banana Orange
주                          
Arizona    13     35      7
Florida    14     52      5
Texas      12     40      9

In [23]:
df.unstack(level=0)  # level=0(가장 바깥쪽)인 index를 컬럼명으로 뺸다.(기본:가장 안쪽)

개수              
주      Arizona Florida Texas
과일명                         
Apple       13      14    12
Banana      35      52    40
Orange       7       5     9

# [melt()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html) - 컬럼명을 컬럼의 값으로 변환한다.

- stack()과 같이 컬럼 명을 단일 컬럼의 값으로 변환한다.
- 변환할 컬럼들을 지정할 수 있어 stack()보다 더 유연하다.


## 매개변수
- id_vars: 값으로 변환하지 않고 그대로 유지하고자 하는 컬럼명(열이름)들의 리스트
    - *식별변수*라고도 한다.
    - 지정한 변수(컬럼)은 같은 열에 남게 되지만, value_vars에 전달된 각 열에 대해 반복적으로 나타난다.
- value_vars: 단일 컬럼의 값으로 변경하고자 하는 컬럼명 리스트
    - value_vars에 지정한 컬럼이 value가 되고 그 컬럼의 값들은 다른 컬럼으로 생성된다.
    - id_vars와 value_vars에 **지정 안된** 컬럼은 제거된다.
        - 제거 되지 않고 **단독 컬럼으로 유지되길 바라는 컬럼은 id_vars**로 지정한다.
- var_name: value_vars로 컬럼의 값으로 바뀐 컬럼의 이름 지정(지정 안하면 컬럼명은 **variable**)
- value_name: value_vars로 값이된 컬럼들이 원래 가지고 있던 값들로 구성된 컬럼의 이름 지정(지정안하면 컬럼명은 **value**)


>
> - melt 한 경우 **원래 index명은 무시된다.** => RangeIndex로 대체된다.
>    - index를 유지하려면 **reset_index**를 이용해 value만든 다음에 melt()를 호출한다.
        

In [25]:
state_fruits.reset_index(inplace=True)
state_fruits

,State,Apple,Orange,Banana
0,Texas,12,9,40
1,Arizona,13,7,35
2,Florida,14,5,52


In [28]:
# 유지할 컬럼 - State
# 컬럼값으로 옮길 컬럼 - 과일명
state_fruits.melt(id_vars='State',
                  value_vars=['Apple','Orange','Banana'],
                  var_name='Fruits',
                  value_name='Count')

,State,Fruits,Count
0,Texas,Apple,12
1,Arizona,Apple,13
2,Florida,Apple,14
3,Texas,Orange,9
4,Arizona,Orange,7
5,Florida,Orange,5
6,Texas,Banana,40
7,Arizona,Banana,35
8,Florida,Banana,52


In [29]:
state_fruits.melt(id_vars='State',value_vars=['Apple','Orange'])
# Banana는 id_vars나 value_vars에 지정하지 않음
# id_vars나 value_vars에 지정하지 않은 컬럼은 제거됨

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,13
2,Florida,Apple,14
3,Texas,Orange,9
4,Arizona,Orange,7
5,Florida,Orange,5


In [32]:
# id_vars만 지정하면 나머지 컬럼은 컬럼값으로 모두 빠짐
state_fruits_melt=state_fruits.melt(id_vars='State', var_name='Fruits', value_name='Count')
state_fruits_melt

,State,Fruits,Count
0,Texas,Apple,12
1,Arizona,Apple,13
2,Florida,Apple,14
3,Texas,Orange,9
4,Arizona,Orange,7
5,Florida,Orange,5
6,Texas,Banana,40
7,Arizona,Banana,35
8,Florida,Banana,52


# pivot - index, column, value가 될 컬럼들을 지정해 재구조화
- 데이터프레임 재구조화가 목적인데 melt된 것을 원상복구 시킬때도 사용할 수 있다.

## pivot 매개변수
- **각 매개변수의 값은 단일 문자열로 컬럼명을 준다.**
- index: 문자열(리스트안됨). 행이름으로 사용할 컬럼 -> 열이 index로 이동하는 형태가 된다.
- columns: 문자열(리스트안됨). 컬럼명으로 사용할 컬럼
    - **index와 columns 는 여러개 지정 안됨. 오직 하나만 지정 가능**
- values : Value에 올 컬럼명

In [43]:
df=state_fruits_melt.pivot(index='State',columns='Fruits',values='Count')
df

Fruits,Apple,Banana,Orange
State,,,
Arizona,13,35,7
Florida,14,52,5
Texas,12,40,9


In [40]:
df['Apple']

State
Arizona    13
Florida    14
Texas      12
Name: Apple, dtype: int64

In [44]:
df.loc['Texas','Banana'], df.loc['Arizona','Orange']

(40, 7)